### Word In N Gram Or Sentence

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

In [ ]:
#import modin.pandas as pd
import pandas as pd
import numpy as np
import re
import nltk

In [ ]:
#!pip install unicodedata2 

In [ ]:
word_num = 1000
data_kind = "fivegram" # threegram, fourgram

In [ ]:
df = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/French/Result/Sentence/Merge/{data_kind.capitalize()}_Merge_Result2.csv")
df

In [ ]:
df_word = pd.read_csv("/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/French/Result/Word/Merge/French_Word_Merge_Result2.csv")
df_word

In [ ]:
df_word = df_word.head(word_num)

In [ ]:
word_list = df_word["word"].values.tolist()
#print(word_list)

In [ ]:
sent_ngram_list  = df.iloc[:,0].values.tolist()
#print(sentencence_list)

In [ ]:
len(sent_ngram_list )

In [ ]:
data_kind = "sentence"
resultlist = []
for i in sent_ngram_list:
    mergelist = []
    try:
        word = i.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(i)
                                                    
#print(resultlist)

In [ ]:
len (resultlist)

In [ ]:
df_result = pd.DataFrame(resultlist)
df_result.head()

In [ ]:
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_result

In [ ]:
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind}")
df_merge

In [ ]:
df_merge = df_merge.sort_values(by="frequency", ascending=False)
df_merge

In [ ]:
df_merge.to_csv(f"{data_kind.capitalize()}_With_{word_num}_Word.csv", index=False)

In [ ]:
df_word.to_excel(f"Word_{word_num}.xlsx", sheet_name='Word', index=False)

#### Multi Process

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
word_num = 300
df_word = pd.read_csv("/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/NLTK/Result/Word/Merge_Word/Word_Merge.csv") # word merge dataframe
df_word = df_word.head(word_num)
df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
295,mutlu,254555
296,ondan,252864
297,eminim,252546
298,insan,252056


In [4]:
word_list = df_word.iloc[:,0].values.tolist()

In [5]:
data_kind = "threegram" # twogram, fourgram
df = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/Result/N Gram/Merge//{data_kind.capitalize()}_Merge.csv") # sentence, n gram dataframe
df

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [6]:
d_list  = df.iloc[:,0].values.tolist()

In [7]:
resultlist = []

In [8]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [9]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,hayır hayır hayır,69443
...,...,...
171902,eğer ona hemen,10
171900,onu benim de,10
171899,eğer ona küçük,10
171898,eğer ona yine,10


In [10]:
df_merge_result.to_csv(f"{data_kind.capitalize()}_With_{word_num}_Word.csv", index=False)

In [ ]:
df_word.to_excel(f"Word_{word_num}.xlsx", sheet_name='Word', index=False)